In [ ]:
import os
import torchvision.models as models 
import torch.nn as nn
import torch
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torchvision
from glob import glob
from torchinfo import summary
import numpy as np
import torch.functional as F
import torchvision.transforms as T
from tqdm.notebook import tqdm
import torchmetrics
device0 = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
Type_class='Type_2'
batch_size=16

fit generator dataload

In [ ]:
class CustomGeneratorDataset(Dataset):
    def __init__(self, data_path,Type_class ,transform=None, target_transform=None):
        self.data_list=glob(data_path+Type_class+'/**/*.csv')
        
    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        signal_csv=pd.read_csv(self.data_list[idx])['PPG'].to_numpy()
        signal_tensor=torch.reshape(torch.tensor(signal_csv),(1,1000))
        self.data_list[idx].find('positive')
        AF_signal_label=0
        if self.data_list[idx].find('positive')!=-1:
            AF_signal_label=1
        else:
            AF_signal_label=0
        AF_signal_label=torch.tensor([AF_signal_label])
        return signal_tensor,AF_signal_label
    
train_csv_path ='../../data/AF/Waveform_30/train/'
test_csv_path ='../../data/AF/Waveform_30/test/'
train_dataset=CustomGeneratorDataset(train_csv_path,Type_class)
test_dataset=CustomGeneratorDataset(test_csv_path ,Type_class)
dataset_size = len(test_dataset)
test_size = int(dataset_size * 0.5)
validation_size = dataset_size-test_size
validation_dataset, test_dataset = random_split(test_dataset, [validation_size, test_size])
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

fit dataload

In [ ]:
#fit
class CustomDataset(Dataset):
    def __init__(self, data,label ,transform=None, target_transform=None):
        self.PPG_data=data
        self.PPG_label=label
    def __len__(self):
        return len(self.PPG_data)

    def __getitem__(self, idx):
        signal_tensor=self.PPG_data[idx]
        AF_signal_label=self.PPG_label[idx]
        return signal_tensor,AF_signal_label
    
train_csv_path ='../../data/AF/Waveform_30/train/'
test_csv_path ='../../data/AF/Waveform_30/test/'

train_data_list=glob(train_csv_path+Type_class+'/**/*.csv')
test_data_list=glob(test_csv_path+Type_class+'/**/*.csv')
train_data=torch.empty(len(train_data_list),1,3000)
train_label=torch.empty(len(train_data_list),1)
test_data=torch.empty(len(test_data_list),1,3000)
test_label=torch.empty(len(test_data_list),1)
for i in range(len(train_data_list)):
    signal_csv=pd.read_csv(train_data_list[i])['PPG'].to_numpy()
    train_data_list[i].find('positive')
    AF_signal_label=0
    if train_data_list[i].find('positive')!=-1:
        AF_signal_label=1
    else:
        AF_signal_label=0
    train_data[i]=torch.tensor(signal_csv)
    train_label[i]=torch.tensor([AF_signal_label])
    
for i in range(len(test_data_list)):
    signal_csv=pd.read_csv(test_data_list[i])['PPG'].to_numpy()
    test_data_list[i].find('positive')
    AF_signal_label=0
    if test_data_list[i].find('positive')!=-1:
        AF_signal_label=1
    else:
        AF_signal_label=0
    test_data[i]=torch.tensor(signal_csv)
    test_label[i]=torch.tensor([AF_signal_label])

train_dataset=CustomDataset(train_data,train_label)
test_dataset=CustomDataset(test_data ,test_label)
dataset_size = len(test_dataset)
test_size = int(dataset_size * 0.5)
validation_size = dataset_size-test_size
validation_dataset, test_dataset = random_split(test_dataset, [validation_size, test_size])
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

model

In [ ]:
from sympy import flatten


class CNN1D(nn.Module):
    def __init__(self):
        super(CNN1D,self).__init__()
        self.layer1 = torch.nn.Sequential(
        torch.nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3),
        torch.nn.MaxPool1d(kernel_size=2,stride=2),
        torch.nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3),
        torch.nn.MaxPool1d(kernel_size=2,stride=2) 
        ,
        torch.nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3),
        torch.nn.MaxPool1d(kernel_size=2,stride=2) 
        )
        self.lstm1=nn.LSTM(input_size=373, hidden_size=128,bidirectional=True,bias = True, 
                            batch_first = True, num_layers=64)
        self.layer2=torch.nn.Sequential(torch.nn.Conv1d(in_channels=128, out_channels=64, kernel_size=3),
                                        torch.nn.MaxPool1d(kernel_size=2,stride=2))
        self.lstm2=nn.LSTM(input_size=63, hidden_size=1,bidirectional=True,bias = True, 
                            batch_first = True, num_layers=32)
        
        self.layer3=torch.nn.Sequential(nn.Linear(in_features = 64, out_features = 256),
                                        nn.LeakyReLU(),
                                        nn.Dropout1d(0.5),
                                        nn.Linear(in_features = 256, out_features = 128),
                                        nn.LeakyReLU(),
                                        nn.Dropout1d(0.5),
                                        nn.Linear(in_features = 128, out_features = 1))
    def forward(self,x):
        x=self.layer1(x)
        self.lstm2.flatten_parameters()
        _, (x, _) = self.lstm1(x)
        x=x.transpose(1,0)
        x=self.layer2(x)
        
        # Shape : (B, S, H) // H = hidden_size => (B, 10, 50)
        self.lstm2.flatten_parameters()
        # Shape : (B, S, H) // H = hidden_size => (B, 10, 50)
        _, (x, _) = self.lstm2(x)
        x=x.transpose(1,0)
        x=x.squeeze(dim=2) 
        x=self.layer3(x)
        
        
        return x


    
accuracy = torchmetrics.Accuracy(task="binary", num_classes=1).to(device0)
model = CNN1D().to(device0)
criterion = nn.BCEWithLogitsLoss().to(device0)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)
summary(model,(batch_size,1,3000))

Train

In [6]:
MIN_loss=5000
train_loss_list=[]
val_loss_list=[]
train_acc_list=[]
val_acc_list=[]
for epoch in range(10000):
    train=tqdm(train_dataloader)
    count=0
    running_loss = 0.0
    acc_loss=0
    for x, y in train:
        model.train()
        y = y.to(device0).float()
        count+=1
        x=x.to(device0).float()
        optimizer.zero_grad()  # optimizer zero 로 초기화
        predict = model(x).to(device0)
        cost = criterion(predict, y) # cost 구함
        acc=accuracy(predict, y)
        cost.backward() # cost에 대한 backward 구함
        optimizer.step() 
        running_loss += cost.item()
        acc_loss+=acc
        train.set_description(f"epoch: {epoch+1}/{10000} Step: {count+1} loss : {running_loss/count:.4f} accuracy: {acc_loss/count:.4f}")
    train_loss_list.append((running_loss/count))
    train_acc_list.append((acc_loss/count).cpu().detach().numpy())
#validation
    val=tqdm(validation_dataloader)
    model.eval()
    count=0
    val_running_loss=0.0
    acc_loss=0
    with torch.no_grad():
        for x, y in val:
            y = y.to(device0).float()
            count+=1
            x=x.to(device0).float()
            
            predict = model(x).to(device0)
            acc=accuracy(predict, y)
            cost = criterion(predict, y)
            val_running_loss+=cost.item()
            acc_loss+=acc
            val.set_description(f"Validation epoch: {epoch+1}/{10000} Step: {count+1} loss : {val_running_loss/count:.4f}  accuracy: {acc_loss/count:.4f}")
        val_loss_list.append((val_running_loss/count))
        val_acc_list.append((acc_loss/count).cpu().detach().numpy())
    if epoch%10==1:
        plt.figure(figsize=(10,5))
        plt.subplot(1, 2, 1) 
        plt.title('loss_graph')
        plt.plot(np.arange(epoch+1),train_loss_list,label='train_loss')
        plt.plot(np.arange(epoch+1),val_loss_list,label='validation_loss')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.legend()
        plt.subplot(1, 2, 2)  
        plt.title('acc_graph')
        plt.plot(np.arange(epoch+1),train_acc_list,label='train_acc')
        plt.plot(np.arange(epoch+1),val_acc_list,label='validation_acc')
        plt.xlabel('epoch')
        plt.ylabel('accuracy')
        plt.legend()
        plt.show()
        
    if MIN_loss<(val_running_loss/count):
        torch.save(model.state_dict(), '../../model/Waveform/'+Type_class+'/BiLSTM_CNN_1D_Type2_call_back.pt')
        MIN_loss=(val_running_loss/count)

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/242 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/242 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/242 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/242 [00:00<?, ?it/s]